# Extract OSM Power Infrastructure Data for Arctic Analysis

This notebook extracts and standardizes power infrastructure data from OpenStreetMap (OSM) for Arctic regions using the [`osm-flex`](https://github.com/osm-flex/osm-flex) Python library.

## About osm-flex

`osm-flex` is a powerful Python library for extracting and processing OpenStreetMap data. It provides:
- Fast PBF file processing
- Flexible feature filtering based on OSM tags
- Automatic geometry handling
- Integration with GeoPandas for spatial analysis

Learn more: https://github.com/osm-flex/osm-flex (GPL-3.0 License)

## What This Workflow Extracts

Using `osm-flex`, this notebook processes OSM data from four Arctic regions:
- **Alaska**: Power infrastructure across the state
- **Canada**: Comprehensive grid data including northern territories
- **Greenland**: Limited but critical coastal infrastructure
- **Russia (Far Eastern Federal District)**: Eastern Arctic grid systems

### Infrastructure Types

1. **Substations** (`power=substation`): High-voltage switching stations, converter stations
2. **Transmission Lines** (`power=line|minor_line|cable`): Overhead and underground power lines
3. **Support Structures** (`power=pole|tower`): Transmission towers and distribution poles

## Output

The workflow produces clean, GIS-ready shapefiles with:
- Standardized geometries (points for substations/structures, lines for transmission)
- Arctic-optimized coordinate transformations (EPSG:3413)
- Comprehensive attribute tables
- Quality control and validation

## Data Sources

OSM data accessed via [Geofabrik](https://download.geofabrik.de/) regional extracts:
- Alaska: `north-america/us/alaska-latest.osm.pbf`
- Canada: `north-america/canada-latest.osm.pbf`
- Greenland: `north-america/greenland-latest.osm.pbf`
- Russia: `russia/far-eastern-fed-district-latest.osm.pbf`

**Data License**: OpenStreetMap © OSM contributors, [ODbL](https://opendatacommons.org/licenses/odbl/)

## 1. Environment Setup

Install and configure the required dependencies. This notebook is designed to run in Google Colab.

In [ ]:
# Install condacolab for conda package management in Colab
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Verify condacolab installation
import condacolab
condacolab.check()

In [ ]:
# Install osm-flex and geospatial dependencies
!conda install -y osm-flex
!pip install osm-flex

## 2. Import Libraries

In [ ]:
# Core libraries
import os
import sys
import zipfile
import urllib.request
from pathlib import Path

# Geospatial libraries
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, LineString, MultiPolygon

# OSM extraction
import osm_flex
import osm_flex.download as dl
import osm_flex.extract as ex
import osm_flex.config
import osm_flex.clip as cp

# Enable logging
osm_flex.enable_logs()

print("✓ All libraries imported successfully")

## 3. Utility Functions

In [ ]:
def convert_to_centroids(gdf, buffer_distance=1, target_crs='EPSG:3413'):
    """
    Converts polygon geometries to point centroids for substation locations.
    
    OSM substation data contains mixed geometry types (points and polygons). 
    This function standardizes all geometries to point centroids, which is
    required for least-cost path analysis between substations.
    
    Parameters
    ----------
    gdf : GeoDataFrame
        Input GeoDataFrame with mixed geometry types
    buffer_distance : float, default=1
        Buffer distance for Points/LineStrings in target CRS units (meters)
    target_crs : str, default='EPSG:3413'
        Arctic Polar Stereographic projection for accurate distance calculations
    
    Returns
    -------
    GeoDataFrame
        GeoDataFrame with point geometries representing centroids
    """
    original_crs = gdf.crs
    
    # Reproject to Arctic Polar Stereographic for accurate calculations
    if target_crs:
        gdf = gdf.to_crs(target_crs)
    
    # Buffer small geometries, keep polygons as-is
    gdf['geometry'] = gdf.geometry.apply(
        lambda geom: geom.buffer(buffer_distance) 
        if geom.geom_type in ['Point', 'LineString'] 
        else geom
    )
    
    # Ensure all geometries are valid polygons
    gdf['geometry'] = gdf['geometry'].apply(
        lambda geom: geom 
        if geom.geom_type in ['Polygon', 'MultiPolygon'] 
        else Polygon(geom)
    )
    
    # Calculate centroids
    gdf['geometry'] = gdf.geometry.centroid
    
    # Reproject back to original CRS
    if target_crs:
        gdf = gdf.to_crs(original_crs)
    
    return gdf


def download_osm_data(region_name, url, output_dir='/root/osm/osm_pbf'):
    """
    Download OSM PBF file for a specified region.
    
    Parameters
    ----------
    region_name : str
        Name of the region (e.g., 'alaska', 'canada')
    url : str
        Geofabrik download URL for the OSM PBF file
    output_dir : str, default='/root/osm/osm_pbf'
        Directory to save downloaded files
    
    Returns
    -------
    str
        Path to downloaded file
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Define output path
    output_path = os.path.join(output_dir, f"{region_name}-latest.osm.pbf")
    
    # Download file
    print(f"Downloading {region_name} OSM data...")
    urllib.request.urlretrieve(url, output_path)
    print(f"✓ Downloaded to {output_path}")
    
    return output_path


def extract_power_infrastructure(pbf_path, feature_type='power'):
    """
    Extract power infrastructure features from OSM PBF file.
    
    Parameters
    ----------
    pbf_path : str
        Path to OSM PBF file
    feature_type : str, default='power'
        OSM feature type to extract
    
    Returns
    -------
    GeoDataFrame
        Power infrastructure features with geometry and attributes
    """
    print(f"Extracting {feature_type} features from {pbf_path}...")
    gdf = ex.extract_cis(pbf_path, feature_type)
    print(f"✓ Extracted {len(gdf)} features")
    return gdf


def filter_substations(gdf_power):
    """
    Filter power infrastructure data to extract only substations.
    
    Parameters
    ----------
    gdf_power : GeoDataFrame
        Power infrastructure GeoDataFrame
    
    Returns
    -------
    GeoDataFrame
        Filtered GeoDataFrame containing only substations
    """
    gdf_substations = gdf_power[gdf_power['power'] == 'substation'].copy()
    print(f"✓ Filtered {len(gdf_substations)} substations")
    return gdf_substations


def export_to_shapefile(gdf, output_path, region_name):
    """
    Export GeoDataFrame to shapefile format with OSM attribution.
    
    Adds required ODbL attribution fields to comply with OpenStreetMap license.
    
    Parameters
    ----------
    gdf : GeoDataFrame
        GeoDataFrame to export
    output_path : str
        Directory to save shapefile
    region_name : str
        Region name for file naming
    
    Returns
    -------
    str
        Path to exported shapefile
    
    Notes
    -----
    OSM data is licensed under ODbL and requires attribution.
    See: https://www.openstreetmap.org/copyright
    """
    # Add OSM attribution metadata (required by ODbL license)
    gdf_export = gdf.copy()
    gdf_export['osm_src'] = 'OpenStreetMap'
    gdf_export['osm_lic'] = 'ODbL'
    gdf_export['osm_attr'] = '© OSM contributors'
    
    # Export to shapefile
    Path(output_path).mkdir(parents=True, exist_ok=True)
    output_file = os.path.join(output_path, f"{region_name}_substations.shp")
    gdf_export.to_file(output_file)
    print(f"✓ Exported to {output_file} (with OSM attribution)")
    return output_file


print("✓ Utility functions defined")

## 4. Configuration

Define region-specific download URLs and parameters.

In [ ]:
# Define regions and their Geofabrik URLs
REGIONS = {
    'alaska': 'https://download.geofabrik.de/north-america/us/alaska-latest.osm.pbf',
    'canada': 'https://download.geofabrik.de/north-america/canada-latest.osm.pbf',
    'greenland': 'https://download.geofabrik.de/north-america/greenland-latest.osm.pbf',
    'russia_far_east': 'https://download.geofabrik.de/russia/far-eastern-fed-district-latest.osm.pbf'
}

# Output directories
PBF_DIR = '/root/osm/osm_pbf'
OUTPUT_DIR = '/root/osm/outputs'

print(f"Configured {len(REGIONS)} regions for extraction")

## 5. Extract Power Infrastructure by Region

Process each Arctic region individually. This modular approach allows for:
- Region-specific quality control
- Easier debugging
- Flexible analysis workflows

### 5.1 Alaska

In [ ]:
# Download Alaska OSM data
alaska_pbf = download_osm_data('alaska', REGIONS['alaska'], PBF_DIR)

# Extract all power infrastructure
gdf_ak_power = extract_power_infrastructure(alaska_pbf)

# Display unique power types
print("\nPower infrastructure types in Alaska:")
print(gdf_ak_power['power'].unique())

# Display unique voltage levels
print("\nVoltage levels in Alaska:")
print(gdf_ak_power['voltage'].unique())

In [ ]:
# Filter and process substations
gdf_ak_substations = filter_substations(gdf_ak_power)

# Convert to point centroids
gdf_ak_substations = convert_to_centroids(gdf_ak_substations)

# Display summary
print(f"\nAlaska Substations Summary:")
print(f"Total substations: {len(gdf_ak_substations)}")
print(f"CRS: {gdf_ak_substations.crs}")

# Display first few rows
gdf_ak_substations.head()

In [ ]:
# Export Alaska substations
export_to_shapefile(gdf_ak_substations, OUTPUT_DIR, 'alaska')

### 5.2 Canada

In [ ]:
# Download Canada OSM data
canada_pbf = download_osm_data('canada', REGIONS['canada'], PBF_DIR)

# Extract power infrastructure
gdf_ca_power = extract_power_infrastructure(canada_pbf)

# Filter substations
gdf_ca_substations = filter_substations(gdf_ca_power)

# Convert to centroids
gdf_ca_substations = convert_to_centroids(gdf_ca_substations)

# Summary
print(f"\nCanada Substations Summary:")
print(f"Total substations: {len(gdf_ca_substations)}")

# Export
export_to_shapefile(gdf_ca_substations, OUTPUT_DIR, 'canada')

### 5.3 Greenland

In [ ]:
# Download Greenland OSM data
greenland_pbf = download_osm_data('greenland', REGIONS['greenland'], PBF_DIR)

# Extract power infrastructure
gdf_gl_power = extract_power_infrastructure(greenland_pbf)

# Filter substations
gdf_gl_substations = filter_substations(gdf_gl_power)

# Convert to centroids
gdf_gl_substations = convert_to_centroids(gdf_gl_substations)

# Summary
print(f"\nGreenland Substations Summary:")
print(f"Total substations: {len(gdf_gl_substations)}")

# Export
export_to_shapefile(gdf_gl_substations, OUTPUT_DIR, 'greenland')

### 5.4 Russia (Far Eastern Federal District)

In [ ]:
# Download Russia Far East OSM data
russia_pbf = download_osm_data('russia_far_east', REGIONS['russia_far_east'], PBF_DIR)

# Extract power infrastructure
gdf_ru_power = extract_power_infrastructure(russia_pbf)

# Filter substations
gdf_ru_substations = filter_substations(gdf_ru_power)

# Convert to centroids
gdf_ru_substations = convert_to_centroids(gdf_ru_substations)

# Summary
print(f"\nRussia Far East Substations Summary:")
print(f"Total substations: {len(gdf_ru_substations)}")

# Export
export_to_shapefile(gdf_ru_substations, OUTPUT_DIR, 'russia_far_east')

## 6. Combined Pan-Arctic Dataset

Merge all regional datasets into a single Pan-Arctic power infrastructure database.

In [ ]:
# Combine all regional substation datasets
gdf_panarctic_substations = pd.concat([
    gdf_ak_substations.assign(region='Alaska'),
    gdf_ca_substations.assign(region='Canada'),
    gdf_gl_substations.assign(region='Greenland'),
    gdf_ru_substations.assign(region='Russia_Far_East')
], ignore_index=True)

print(f"\nPan-Arctic Substations Summary:")
print(f"Total substations: {len(gdf_panarctic_substations)}")
print(f"\nSubstations by region:")
print(gdf_panarctic_substations.groupby('region').size())

# Export combined dataset
export_to_shapefile(gdf_panarctic_substations, OUTPUT_DIR, 'panarctic_all')

## 7. Extract Support Structures (Poles & Towers)

Extract and standardize poles, towers, and portal structures. These support structures are critical for understanding grid density and construction patterns.

In [ ]:
def filter_support_structures(gdf_power):
    """
    Filter power infrastructure to extract poles, towers, and portals.
    
    Parameters
    ----------
    gdf_power : GeoDataFrame
        Power infrastructure GeoDataFrame
    
    Returns
    -------
    GeoDataFrame
        Filtered GeoDataFrame containing support structures
    """
    structures = gdf_power[gdf_power['power'].isin(['pole', 'tower', 'portal'])].copy()
    print(f"✓ Filtered {len(structures)} support structures")
    return structures

# Extract support structures for each region
gdf_ak_structures = filter_support_structures(gdf_ak_power)
gdf_ca_structures = filter_support_structures(gdf_ca_power)
gdf_gl_structures = filter_support_structures(gdf_gl_power)
gdf_ru_structures = filter_support_structures(gdf_ru_power)

# Standardize to points (many are small polygons)
gdf_ak_structures = convert_to_centroids(gdf_ak_structures)
gdf_ca_structures = convert_to_centroids(gdf_ca_structures)
gdf_gl_structures = convert_to_centroids(gdf_gl_structures)
gdf_ru_structures = convert_to_centroids(gdf_ru_structures)

print(f"\nSupport Structure Summary by Region:")
print(f"Alaska: {len(gdf_ak_structures):,}")
print(f"Canada: {len(gdf_ca_structures):,}")
print(f"Greenland: {len(gdf_gl_structures):,}")
print(f"Russia: {len(gdf_ru_structures):,}")

In [ ]:
# Combine all structures
gdf_panarctic_structures = pd.concat([
    gdf_ak_structures.assign(region='Alaska'),
    gdf_ca_structures.assign(region='Canada'),
    gdf_gl_structures.assign(region='Greenland'),
    gdf_ru_structures.assign(region='Russia_Far_East')
], ignore_index=True)

print(f"\nTotal support structures: {len(gdf_panarctic_structures):,}")
print(f"\nStructure types:")
print(gdf_panarctic_structures['power'].value_counts())

# Export
export_to_shapefile(gdf_panarctic_structures, OUTPUT_DIR, 'panarctic_poles_towers')

## 8. Extract Transmission Lines

Extract existing transmission line data. Unlike substations and structures, lines maintain their linestring geometry.

In [ ]:
def filter_transmission_lines(gdf_power):
    """
    Filter power infrastructure to extract transmission lines.
    
    Parameters
    ----------
    gdf_power : GeoDataFrame
        Power infrastructure GeoDataFrame
    
    Returns
    -------
    GeoDataFrame
        Filtered GeoDataFrame containing only transmission lines
    """
    gdf_lines = gdf_power[gdf_power['power'].isin(['line', 'minor_line', 'cable'])].copy()
    print(f"✓ Filtered {len(gdf_lines)} transmission line segments")
    return gdf_lines

# Extract transmission lines for each region
gdf_ak_lines = filter_transmission_lines(gdf_ak_power)
gdf_ca_lines = filter_transmission_lines(gdf_ca_power)
gdf_gl_lines = filter_transmission_lines(gdf_gl_power)
gdf_ru_lines = filter_transmission_lines(gdf_ru_power)

# Combine
gdf_panarctic_lines = pd.concat([
    gdf_ak_lines.assign(region='Alaska'),
    gdf_ca_lines.assign(region='Canada'),
    gdf_gl_lines.assign(region='Greenland'),
    gdf_ru_lines.assign(region='Russia_Far_East')
], ignore_index=True)

print(f"\nTotal transmission line segments: {len(gdf_panarctic_lines)}")

# Export
export_to_shapefile(gdf_panarctic_lines, OUTPUT_DIR, 'panarctic_lines')

## 8. Summary Statistics

In [ ]:
# Generate summary statistics
summary_stats = pd.DataFrame({
    'Region': ['Alaska', 'Canada', 'Greenland', 'Russia_Far_East', 'Total'],
    'Substations': [
        len(gdf_ak_substations),
        len(gdf_ca_substations),
        len(gdf_gl_substations),
        len(gdf_ru_substations),
        len(gdf_panarctic_substations)
    ],
    'Transmission_Lines': [
        len(gdf_ak_lines),
        len(gdf_ca_lines),
        len(gdf_gl_lines),
        len(gdf_ru_lines),
        len(gdf_panarctic_lines)
    ]
})

print("\n" + "="*60)
print("PAN-ARCTIC POWER INFRASTRUCTURE EXTRACTION SUMMARY")
print("="*60)
print(summary_stats.to_string(index=False))
print("="*60)

# Save summary to CSV
summary_path = os.path.join(OUTPUT_DIR, 'extraction_summary.csv')
summary_stats.to_csv(summary_path, index=False)
print(f"\n✓ Summary saved to {summary_path}")

## 9. Next Steps

The extracted OSM data provides the foundation for:

1. **Grid Boundary Delineation**: Analyze clustering patterns of transmission infrastructure to identify discrete grid regions
2. **Google Earth Engine Integration**: Import substations as assets for friction surface development and least-cost path analysis
3. **Cost Estimation**: Combine with land cover, elevation, permafrost, and accessibility data to model transmission corridor costs

### Data Quality Notes

- OSM data completeness varies by region
- Canada has the most comprehensive coverage (5,972 substations)
- Greenland has limited coverage (15 substations) due to sparse infrastructure
- Russia Far East data represents 2024 OSM snapshot and may require validation
- Mixed geometry types have been standardized to point centroids for consistency

### Citation

If you use this data extraction workflow, please cite:

```
Trochim, E., Lockhard, D., Foster, M., Zhu, L., Bryant, L., & Jacobs, N. (2025). 
Pan-Arctic Energy Distribution Networks: Interconnection Cost Estimations and Route Modeling.
```

OSM data © OpenStreetMap contributors, available under the Open Database License (ODbL).